# [Cross-Validation](https://www.kaggle.com/dansbecker/cross-validation/code)

Machine learning is an iterative process.  
You will face choices about predictive variables to use, what types of models to use, what arguments to supply those models, and many other options.  
We make these choices in a data-driven way by measuring model quality of various alternatives.  
You've already learned to use `train_test_split` to split the data, so you can measure model quality on the test data.  
Cross-validation extends this approach to model scoring (or "model validation.")  
Compared to `train_test_split`, cross-validation gives you a more reliable measure of your model's quality, though it takes longer to run.

### The shortcomings of train-test split:

Imagine you have a dataset with 5000 rows.  
The `train_test_split` function has an argument for `test_size` that you can use to decide how many rows go to the training set and how many go to the test set.   
The larger the test set, the more reliable your measures of model quality will be.  
At an extreme, you could imagine having only 1 row of data in the test set.  
If you compare alternative models, which one makes the best predictions on a single data point will be mostly a matter of luck.  
You will typically keep about 20% as a test dataset.  
But even with 1000 rows in the test set, there's some random chance in determining model scores.  
A model might do well on one set of 1000 rows, even if it would be inaccurate on a different 1000 rows.  
The larger the test set, the less randomness (aka "noise") there is in our measure of model quality.  
But we can only get a large test set by removing data from our training data, and smaller training datasets mean worse models.  
In fact, the ideal modeling decisions on small datasets typically aren't the best modeling decisions on large datasets.

## The Cross-Validation Procedure